In [ ]:
import requests
import pandas as pd
import datetime
from psycopg2.extras import execute_values
from db import get_db_config

In [ ]:
 -----------------------------# Configuração da tabela SQL
# ----------------------------
# CREATE TABLE dados_api(
#     cidade VARCHAR(100) NOT NUL,
#     bairro VARCHAR(100) NOT NLL,
#     cep VARCHAR(9) NOT ULL,
#     data_registro DATE NO NULL
# );


In [3]:
url = 'https://viacep.com.br/ws/BA/Feira de Santana/SIM/json/'

In [7]:
response = requests.get(url)

NameError: name 'requests' is not defined

### Transformando o JSON retornado em DataFrame

In [ ]:
df = pd.DataFrame(response.json())

### Removendo colunas desnecessárias

In [ ]:
df = df.drop(columns=['complemento', 'unidade','estado','ibge', 'gia', 'ddd', 'siafi', 'uf', 'regiao', 'logradouro'])

### Adicionando coluna com a data e hora da coleta

In [ ]:
df['data_registro'] = datetime.datetime.now()

### Convertendo nome da cidade para maiúsculo

In [ ]:
df['localidade'] = df['localidade'].str.upper()

### Reorganizando colunas para manter ordem desejada

In [ ]:
df = df.reindex(columns=['localidade', 'bairro', 'cep', 'data_registro'])

### Exibindo o DataFrame final

In [ ]:
print("DataFrame preparado para inserção no banco:")
print(df)

### Convertendo DataFrame em lista de tuplas

In [ ]:
list_df = df.to_records(index=False).tolist()

### Exibindo a lista de tuplas

In [ ]:
print(list_df)

### Conectando ao banco

In [ ]:
con = get_db_config()
cur = con.cursor()

### Inserido os dados na tabela do banco

In [ ]:
try:
    query = '''
    INSERT INTO dados_api (cidade, bairro, cep, data_registro)
    VALUES %s
    '''

    # Insere os dados vários registros de uma vez
    execute_values(cur, query, list_df)

    # Confirmando transação
    con.commit()

    print('Dados inseridos com sucesso!')

    # Fechando cursor e conexão
    cur.close()
    con.close()

except Exception as e:
    print(e)

In [ ]:
# try:
#     for i, d in df.iterrows():
#         query = '''
#         INSERT INTO dados_api (cidade, bairro, cep, data_registro)
#         VALUES (%s, %s, %s, %s, %s)
#         '''
#
#         # Insere os dados vários registros de uma vez
#         execute_values(cur, (d['localidade'], d['bairro'], d['cep'], d['responsavel']))
#
#     # Confirmando transação
#     con.commit()
#     print('Dados inseridos com sucesso!')
#
#     # Fechando cursor e conexão
#     cur.close()
#     con.close()
#
# except Exception as e:
#     print(e)

In [ ]:
# try:
#     select_query = '''
#     SELECT cidade, bairro, cep, data_registro
#     FROM dados_api
#     ORDER BY data_registro DESC
#     LIMIT 10;
#     '''

#     cur.execute(select_query)
#     rows = cur.fetchall()

#     # Transformando resultado em DataFrame para visualizar melhor
#     df_select = pd.DataFrame(rows, columns=['cidade', 'bairro', 'cep', 'data_registro'])

#     print("\nÚltimos registros no banco:")
#     print(df_select)

#     cur.close()
#     con.close()

# except Exception as e:
#     print("Erro ao consultar dados:", e)